In [1]:
#@title Install requirements. { display-mode: "form" }
# Install requirements
!pip install torch transformers sentencepiece h5py

In [2]:
#@title Set up working directories and download files/checkpoints. { display-mode: "form" }
# Create directory for storing model weights (2.3GB) and example sequences.
# Here we use the encoder-part of ProtT5-XL-U50 in half-precision (fp16) as
# it performed best in our benchmarks (also outperforming ProtBERT-BFD).
# Also download secondary structure prediction checkpoint to show annotation extraction from embeddings
!mkdir protT5 # root directory for storing checkpoints, results etc
!mkdir protT5/protT5_checkpoint # directory holding the ProtT5 checkpoint
!mkdir protT5/sec_struct_checkpoint # directory storing the supervised classifier's checkpoint
!mkdir protT5/output # directory for storing your embeddings & predictions
!wget -nc -P protT5/ https://rostlab.org/~deepppi/example_seqs.fasta
# Huge kudos to the bio_embeddings team here! We will integrate the new encoder, half-prec ProtT5 checkpoint soon
!wget -nc -P protT5/sec_struct_checkpoint http://data.bioembeddings.com/public/embeddings/feature_models/t5/secstruct_checkpoint.pt

mkdir: cannot create directory ‘protT5’: File exists
mkdir: cannot create directory ‘protT5/protT5_checkpoint’: File exists
mkdir: cannot create directory ‘protT5/sec_struct_checkpoint’: File exists
mkdir: cannot create directory ‘protT5/output’: File exists
File ‘protT5/example_seqs.fasta’ already there; not retrieving.

File ‘protT5/sec_struct_checkpoint/secstruct_checkpoint.pt’ already there; not retrieving.



In [3]:
import os
os.getcwd()
os.listdir()

['.sparkmagic',
 '.ipynb_checkpoints',
 'protT5',
 '.virtual_documents',
 '.Trash-1000',
 'PDB_homo.csv',
 'embed_ProtT5 (1).ipynb',
 'lost+found',
 'embeddings.csv',
 'PDB_transcrit.csv',
 'PDB_prot.csv']

In [4]:
os.chdir("/")

In [5]:
os.chdir("home/ec2-user/SageMaker/protT5")

In [6]:
# In the following you can define your desired output. Current options:
# per_residue embeddings
# per_protein embeddings
# secondary structure predictions

# Replace this file with your own (multi-)FASTA
# Headers are expected to start with ">";

# whether to retrieve embeddings for each residue in a protein
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = False
per_residue_path = "./protT5/output/per_residue_embeddings.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = True
per_protein_path = "./protT5/output/per_protein_embeddings.h5" # where to store the embeddings

# whether to retrieve secondary structure predictions
# This can be replaced by your method after being trained on ProtT5 embeddings
sec_struct = False
sec_struct_path = "./protT5/output/ss3_preds.fasta" # file for storing predictions

# make sure that either per-residue or per-protein embeddings are stored
assert per_protein is True or per_residue is True or sec_struct is True, print(
    "Minimally, you need to active per_residue, per_protein or sec_struct. (or any combination)")


In [7]:
#@title Import dependencies and check whether GPU is available. { display-mode: "form" }
from transformers import T5EncoderModel, T5Tokenizer
import torch
import h5py
import time
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using {}".format(device))

Using cuda:0


In [8]:
#@title Network architecture for secondary structure prediction. { display-mode: "form" }
# Convolutional neural network (two convolutional layers) to predict secondary structure
class ConvNet( torch.nn.Module ):
    def __init__( self ):
        super(ConvNet, self).__init__()
        # This is only called "elmo_feature_extractor" for historic reason
        # CNN weights are trained on ProtT5 embeddings
        self.elmo_feature_extractor = torch.nn.Sequential(
                        torch.nn.Conv2d( 1024, 32, kernel_size=(7,1), padding=(3,0) ), # 7x32
                        torch.nn.ReLU(),
                        torch.nn.Dropout( 0.25 ),
                        )
        n_final_in = 32
        self.dssp3_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 3, kernel_size=(7,1), padding=(3,0)) # 7
                        )

        self.dssp8_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 8, kernel_size=(7,1), padding=(3,0))
                        )
        self.diso_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 2, kernel_size=(7,1), padding=(3,0))
                        )


    def forward( self, x):
        # IN: X = (B x L x F); OUT: (B x F x L, 1)
        x = x.permute(0,2,1).unsqueeze(dim=-1)
        x         = self.elmo_feature_extractor(x) # OUT: (B x 32 x L x 1)
        d3_Yhat   = self.dssp3_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 3)
        d8_Yhat   = self.dssp8_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 8)
        diso_Yhat = self.diso_classifier(  x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 2)
        return d3_Yhat, d8_Yhat, diso_Yhat

In [9]:
#@title Load the checkpoint for secondary structure prediction. { display-mode: "form" }
def load_sec_struct_model():
    checkpoint_dir="./protT5/sec_struct_checkpoint/secstruct_checkpoint.pt"
    state = torch.load( checkpoint_dir )
    model = ConvNet()
    model.load_state_dict(state['state_dict'])
    model = model.eval()
    model = model.to(device)
    print('Loaded sec. struct. model from epoch: {:.1f}'.format(state['epoch']))

    return model

In [10]:
#@title Load encoder-part of ProtT5 in half-precision. { display-mode: "form" }
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50)
def get_T5_model():
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

    return model, tokenizer

In [11]:
import os

os.chdir("/home/ec2-user/SageMaker/")

In [12]:

import csv 

homo = {}
prot = {}
transcrit = {}

with open('PDB_homo.csv', mode='r') as infile:
    reader = csv.reader(infile)
    homo = {rows[0]:rows[1] for rows in reader}

with open('PDB_prot.csv', mode='r') as infile:
    reader = csv.reader(infile)
    prot = {rows[0]:rows[1] for rows in reader}
    
with open('PDB_transcrit.csv', mode='r') as infile:
    reader = csv.reader(infile)
    transcrit = {rows[0]:rows[1] for rows in reader}

del homo["PDB_File_Name"]
del prot["PDB_File_Name"]
del transcrit["PDB_File_Name"]

seqs = {}

seqs.update(homo)
seqs.update(prot)
seqs.update(transcrit)

print(len(seqs))


81


In [13]:
#@title Generate embeddings. { display-mode: "form" }
# Generate embeddings via batch-processing
# per_residue indicates that embeddings for each residue in a protein should be returned.
# per_protein indicates that embeddings for a whole protein should be returned (average-pooling)
# max_residues gives the upper limit of residues within one batch
# max_seq_len gives the upper sequences length for applying batch-processing
# max_batch gives the upper number of sequences per batch
def get_embeddings( model, tokenizer, seqs, per_residue, per_protein, sec_struct,
                   max_residues=4000, max_seq_len=1000, max_batch=100 ):

    if sec_struct:
      sec_struct_model = load_sec_struct_model()

    results = {"residue_embs" : dict(),
               "protein_embs" : dict(),
               "sec_structs" : dict()
               }

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            if sec_struct: # in case you want to predict secondary structure from embeddings
              d3_Yhat, d8_Yhat, diso_Yhat = sec_struct_model(embedding_repr.last_hidden_state)


            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                # slice off padding --> batch-size x seq_len x embedding_dim
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if sec_struct: # get classification results
                    results["sec_structs"][identifier] = torch.max( d3_Yhat[batch_idx,:s_len], dim=1 )[1].detach().cpu().numpy().squeeze()
                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results

In [14]:
#@title Write embeddings to disk. { display-mode: "form" }
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "w") as hf:
        for sequence_id, embedding in emb_dict.items():
            # noinspection PyUnboundLocalVariable
            hf.create_dataset(sequence_id, data=embedding)
    return None

In [15]:
#@title Write predictions to disk. { display-mode: "form" }
def write_prediction_fasta(predictions, out_path):
  class_mapping = {0:"H",1:"E",2:"L"}
  with open(out_path, 'w+') as out_f:
      out_f.write( '\n'.join(
          [ ">{}\n{}".format(
              seq_id, ''.join( [class_mapping[j] for j in yhat] ))
          for seq_id, yhat in predictions.items()
          ]
            ) )
  return None

In [16]:
import os 
os.chdir("/home/ec2-user/SageMaker")

In [17]:
os.listdir()
# os.chdir("")

['.sparkmagic',
 '.ipynb_checkpoints',
 'protT5',
 '.virtual_documents',
 '.Trash-1000',
 'PDB_homo.csv',
 'embed_ProtT5 (1).ipynb',
 'lost+found',
 'embeddings.csv',
 'PDB_transcrit.csv',
 'PDB_prot.csv']

In [18]:
os.listdir("protT5")

['sec_struct_checkpoint',
 '.ipynb_checkpoints',
 'protT5_checkpoint',
 'example_seqs.fasta',
 'output']

In [19]:
import torch
use_cuda = torch.cuda.is_available()

In [20]:
print(use_cuda)

True


In [21]:
# Load the encoder part of ProtT5-XL-U50 in half-precision (recommended)
model, tokenizer = get_T5_model()

# Load example fasta.
# seqs already defined

# Compute embeddings and/or secondary structure predictions
results = get_embeddings( model, tokenizer, seqs,
                         per_residue, per_protein, sec_struct)

# Store per-residue embeddings
if per_residue: 
    save_embeddings(results["residue_embs"], per_residue_path)
if per_protein:
    save_embeddings(results["protein_embs"], per_protein_path)
if sec_struct:
    write_prediction_fasta(results["sec_structs"], sec_struct_path)

SyntaxError: invalid syntax (1713951086.py, line 7)

In [ ]:
import csv

def save_dict_to_csv(dictionary, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for key, value in dictionary.items():
            writer.writerow([key, value])


In [ ]:
csv_filename = 'embeddings.csv'

In [ ]:
save_dict_to_csv(results["protein_embs"], csv_filename)

In [ ]:
print(results["protein_embs"])
results = results["protein_embs"]

In [ ]:
scores_train = {'AF-Q92392-F1.pdb': '0.7214553736978107', 'AF-Q04711-F1.pdb': '0.7705930843949318', 'AF-P10978-F1.pdb': '0.7672958493232727', 'AF-A0A5A7R4Q0-F1.pdb': '0.6621873825788498', 'AF-A0A6A3BVT5-F1.pdb': '0.6545961101849874', 'AF-P0CX58-F1.pdb': '0.7776032023959689', 'AF-A0A438G7Q2-F1.pdb': '0.7801340222358704', 'AF-A0A6L2J9H3-F1.pdb': '0.7836335102717081', 'AF-Q03855-F1.pdb': '0.8011221754550933', 'AF-Q12112-F1.pdb': '0.6529602408409119', 'AF-W8C110-F1.pdb': '0.7396126687526703', 'AF-W8AQS8-F1.pdb': '0.6627489262157016', 'AF-Q07163-F1.pdb': '0.7135160702925462', 'AF-P0C2J0-F1.pdb': '0.5091674961149693', 'AF-Q12441-F1.pdb': '0.7415428899583363', 'AF-A0A5N5HA23-F1.pdb': '0.5554636955261231', 'AF-Q12491-F1.pdb': '0.5969818112525073', 'AF-Q12316-F1.pdb': '0.5762644706604382', 'AF-P0CX60-F1.pdb': '0.6828132016318185', 'AF-P0CX63-F1.pdb': '0.6120331010648182', 'AF-A0A438CLY0-F1.pdb': '0.6917162656784057', 'AF-Q12088-F1.pdb': '0.6822161291326795', 'AF-A0A1D8BJE8-F1.pdb': '0.14345672726631165', 'AF-P0C2I9-F1.pdb': '0.5716737593923297', 'AF-A0A2L2YWJ1-F1.pdb': '0.6026824116706848', 'AF-P0CX76-F1.pdb': '0.7592370007187128', 'AF-P0CX69-F1.pdb': '0.7296085506677628', 'AF-Q04706-F1.pdb': '0.7369550606783699'}
scores_test = {'AF-Q99231-F1.pdb': '0.6667538111408552', 'AF-Q12470-F1.pdb': '0.8035197721587287', 'AF-A0A438CP42-F1.pdb': '0.6251117587089539', 'AF-Q03494-F1.pdb': '0.7260885124023144', 'AF-P47099-F1.pdb': '0.6997073143720627', 'AF-Q04214-F1.pdb': '0.7640360531054045', 'AF-W8B9S8-F1.pdb': '0.7416476011276245', 'AF-Q03434-F1.pdb': '0.5454278354133878', 'AF-O13535-F1.pdb': '0.6057061470217175', 'AF-A0A438EZQ7-F1.pdb': '0.5015981793403625', 'AF-Q12472-F1.pdb': '0.612123923169242', 'AF-P0CX64-F1.pdb': '0.739402124285698', 'AF-P25384-F1.pdb': '0.6220023274421692', 'AF-W8AK63-F1.pdb': '0.6100412011146545', 'AF-A0A1D8BJE1-F1.pdb': '0.5655360221862793', 'AF-O13527-F1.pdb': '0.64699936658144', 'AF-Q07793-F1.pdb': '0.6031542474573309', 'AF-P0C2J3-F1.pdb': '0.5656805858016014'}

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

In [ ]:
import numpy as np

score_train_array = []
embeddings_train = []

for key in results:
    # Check if the key exists in both dictionaries
    if key in scores_train:
        # Append score to score_array
        score_train_array.append(float(scores_train[key]))
        # Append numerical array to numerical_array
        embeddings_train.append(results[key])

# Convert numerical_array to numpy array
embeddings_train = np.array(embeddings_train)

# Print or use the arrays as needed
print("Score Train Array:", score_train_array)
print("Embeddings Array:", embeddings_train)

In [ ]:
print(len(score_train_array), len(embeddings_train))

In [ ]:
import numpy as np

score_test_array = []
embeddings_test = []

for key in results:
    # Check if the key exists in both dictionaries
    if key in scores_test:
        # Append score to score_array
        score_test_array.append(float(scores_test[key]))
        # Append numerical array to numerical_array
        embeddings_test.append(results[key])

# Convert numerical_array to numpy array
embeddings_test = np.array(embeddings_test)

# Print or use the arrays as needed
print("Score Test Array:", score_test_array)
print("Embeddings test Array:", embeddings_test)

In [ ]:
print(len(score_test_array), len(embeddings_test))

In [ ]:
# Prepare your data
# Assuming you have embeddings and scores as numpy arrays
# Convert them to PyTorch tensors
embeddings_train = torch.tensor(embeddings_train, dtype=torch.float32)
outputs = torch.tensor(score_train_array, dtype=torch.float32)

print(len(embeddings_train), len(outputs))
print(embeddings_train[0])

In [ ]:
# Define hyperparameters
input_dim = embeddings_train.shape[1]  # Dimension of input embeddings
print(input_dim)
learning_rate = 0.01
num_epochs = 100

step_size = 80  # Step size for learning rate scheduler
gamma = 0.1  # Factor by which to reduce learning rate

In [ ]:
# Define your model
input_dim = 1024  # Dimension of input features
output_dim = 1  # Dimension of output (1 for regression)
model = LinearRegression(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:
# Prepare DataLoader
dataset = TensorDataset(embeddings_train, outputs)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
embeddings_test = torch.tensor(embeddings_test, dtype=torch.float32)
outputs_test = torch.tensor(score_test_array, dtype=torch.float32)

# Create a TensorDataset for the test data
test_dataset = TensorDataset(embeddings_test, outputs_test)

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
import matplotlib.pyplot as plt

# Initialize lists to store training and testing accuracy for plotting
train_accuracy_list = []
test_accuracy_list = []

# Training loop
for epoch in range(num_epochs):
    correct_train_predictions = 0
    total_train_samples = 0
    epoch_train_loss = 0
    
    # Training phase
    for embeddings_batch, scores_batch in dataloader:
        optimizer.zero_grad()
        outputs = model(embeddings_batch)
        
        # Ensure outputs has the correct shape
        outputs = outputs.squeeze().view(-1)
        
        # Calculate accuracy
        predictions = outputs.detach().numpy()
        real_scores = scores_batch.numpy()
        correct_train_predictions += sum(abs(predictions - real_scores) < 0.05)
        total_train_samples += len(predictions)
        
        # Compute MSE loss
        loss = criterion(outputs, scores_batch)
        epoch_train_loss += loss.item()
        
        # Backpropagation
        loss.backward()
        optimizer.step()
    
    # Calculate training accuracy for the epoch
    epoch_train_accuracy = correct_train_predictions / total_train_samples
    train_accuracy_list.append(epoch_train_accuracy)
    
    # Calculate average training loss for the epoch
    epoch_train_loss /= len(dataloader)
    
    # Testing phase
    correct_test_predictions = 0
    total_test_samples = 0
    
    with torch.no_grad():
        for embeddings_test_batch, scores_test_batch in test_dataloader:
            outputs_test = model(embeddings_test_batch)
            outputs_test = outputs_test.squeeze().view(-1)
            
            # Calculate accuracy on test data
            predictions_test = outputs_test.detach().numpy()
            real_scores_test = scores_test_batch.numpy()
            correct_test_predictions += sum(abs(predictions_test - real_scores_test) < 0.1)
            total_test_samples += len(predictions_test)
    
    # Calculate testing accuracy for the epoch
    epoch_test_accuracy = correct_test_predictions / total_test_samples
    test_accuracy_list.append(epoch_test_accuracy)
    
    # Print epoch accuracy and loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {epoch_train_accuracy:.4f}, Training Loss: {epoch_train_loss:.4f}, Testing Accuracy: {epoch_test_accuracy:.4f}")

# Plot training and testing accuracy
plt.plot(range(1, num_epochs+1), train_accuracy_list, label='Training Accuracy')
plt.plot(range(1, num_epochs+1), test_accuracy_list, label='Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Testing Accuracy')
plt.legend()
plt.show()